In [1]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/result_subject2/'

/content/drive/MyDrive


In [2]:
import sys, importlib
#importlib.reload(sys.modules['grasp.config'])
from grasp.config import *
# orveride the data_dir in config file
#data_dir='/content/drive/MyDrive/data/' # googleDrive

In [3]:
data_dir

'/content/drive/MyDrive/data/'

In [4]:
! pip install mne==0.19.2
! pip install torch
! pip install skorch

     |████████████████████████████████| 6.4MB 12.1MB/s 
     |████████████████████████████████| 133kB 13.5MB/s 


In [5]:
import torch
from skorch.helper import predefined_split
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from skorch.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt

from grasp.TSception.Models import TSception2
from grasp.utils import rawData2,SEEGDataset,set_random_seeds,cuda_or_cup
from grasp.config import root_dir, tmp_dir
from grasp.myskorch import plotPrediction, MyRegressor


In [6]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
importlib.reload(sys.modules['grasp.utils'])
from grasp.TSception.Models import TSception,TSception2
from grasp.utils import rawData2,SEEGDataset

In [7]:
# import from another folder
import sys
sys.path.insert(1, '/content/drive/MyDrive/examples')
from IMV_LSTM.networks import IMVTensorLSTM

In [8]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [9]:
device=cuda_or_cup()
seed = 123456789  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed)


GPU computing:   True


In [10]:
%%capture
# suppress the output
sid=2
sampling_rate=1000
traindata, valdata, testdata = rawData2(2,'band','all',move2=True)  # (chns, 15000/15001, 118) (channels, time, trials)
##traindata, valdata, testdata = rawData2('raw','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)


In [11]:
trainx, trainy = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-2, :], valdata[:, -2, :]
testx, testy = testdata[:, :-2, :], testdata[:, -2, :]

samples=trainx.shape[0]
chnNum=trainx.shape[1]
step=50 #ms
T=1000 #ms
totalLen=trainx.shape[2] #ms
batch_size=int((totalLen-T)/step) # 280

train_ds = SEEGDataset(trainx, trainy,T, step)
val_ds = SEEGDataset(valx, valy,T, step)
test_ds = SEEGDataset(testx, testy,T, step)


In [12]:
learning_rate=0.001
num_T = 3 # (6 conv2d layers) * ( 3 kernel each layer)
num_S = 3
dropout=0.5
Lambda = 1e-6
model=TSception2(T, step, sampling_rate,chnNum, num_T, num_S,batch_size,dropout).float()

#if(enable_cuda):
#	model.cuda()
 
#checkpoint = torch.load(result_dir+'checkpoint195.pth')
#net.load_state_dict(checkpoint['state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [13]:
net = MyRegressor(
    model,
    #train_split=predefined_split(valid_set),
    iterator_train__shuffle=True,
    train_split=predefined_split(val_ds),
    max_epochs=200,
    lr=learning_rate,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('plotPrediction', plotPrediction(result_dir)),],
    device = device
)


In [ ]:
net.fit(train_ds,y=None)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        0.3763        0.4462  57.7881
      2        0.3671        0.3466  56.7422
      3        0.3606        0.3461  56.7276
      4        0.3603        0.3450  56.7062
      5        0.3584        0.3451  56.6979
      6        0.3579        0.3459  56.6899
      7        0.3595        0.3456  56.7941
      8        0.3584        0.3489  56.8127
      9        0.3577        0.3476  56.7902
     10        0.3588        0.3478  56.7427
     11        0.3571        0.3468  56.6678
     12        0.3572        0.3468  56.9604
     13        0.3577        0.3477  56.9116
     14        0.3577        0.3485  56.6975
     15        0.3578        0.3473  56.7893
     16        0.3568        0.3475  57.2676
     17        0.3570        0.3493  56.8790
     18        0.3561        0.3502  56.8290
     19        0.3572        0.3500  56.8964
     20        0.3560        0.3479  56.8834
     21   